In [1]:
from nas_201_api import NASBench201API as API

api = API('./data/NAS-Bench-201-v1_1-096897.pth', verbose=False)

In [107]:
index = 0
cifar10_valid_dict = api.get_more_info(index, 'cifar10-valid', 199, hp='200', is_random=False)
cifar10_dict = api.get_more_info(index, 'cifar10', 199, hp='200', is_random=False)  
cifar10_val_acc = cifar10_valid_dict['valid-accuracy']
cifar10_test_acc = cifar10_dict['test-accuracy']
print(cifar10_val_acc)
print(cifar10_test_acc)


cifar100_dict = api.get_more_info(index, 'cifar100', 199, hp='200', is_random=False)
cifar100_dict_val_acc = cifar100_dict['valid-accuracy']
cifar100_dict_test_acc = cifar100_dict['test-accuracy']
print(cifar100_dict_val_acc)
print(cifar100_dict_test_acc)

imagenat16_dict = api.get_more_info(index, 'ImageNet16-120', 199, hp='200', is_random=False)
imagenat16_val_acc = imagenat16_dict['valid-accuracy']
imagenat16_test_acc = imagenat16_dict['test-accuracy']
print(imagenat16_val_acc)
print(imagenat16_test_acc)


81.98266665690103
85.86333333333334
52.70000000406901
52.913333294677734
28.211111075507272
26.633333287556965


In [108]:
info = api.query_meta_info_by_index(index, '200')

cifar10_cost_metrics = info.get_compute_costs('cifar10-valid')
cifar10_flops = cifar10_cost_metrics['flops']
cifar10_params = cifar10_cost_metrics['params']
cifar10_latency = cifar10_cost_metrics['latency']
print(cifar10_flops, cifar10_params, cifar10_latency)

cifar100_cost_metrics = info.get_compute_costs('cifar100')
cifar100_flops = cifar100_cost_metrics['flops']
cifar100_params = cifar100_cost_metrics['params']
cifar100_latency = cifar100_cost_metrics['latency']
print(cifar100_flops, cifar100_params, cifar100_latency)

image16_cost_metrics = info.get_compute_costs('ImageNet16-120')
image16_flops = image16_cost_metrics['flops']
image16_params = image16_cost_metrics['params']
image16_latency = image16_cost_metrics['latency']
print(image16_flops, image16_params, image16_latency)


15.64737 0.129306 0.0139359758611311
15.65322 0.135156 0.013182918230692545
3.91948 0.136456 0.012976543108622235


In [99]:
from xautodl.models import get_cell_based_tiny_net

index = 4
cifar10_config = api.get_net_config(index,'cifar10-valid')
print(cifar10_config)
cifar10_network = get_cell_based_tiny_net(cifar10_config)

from fvcore.nn import FlopCountAnalysis, flop_count_table, flop_count_str, parameter_count, parameter_count_table
import torch
inputs = torch.randn(1,3,32,32)
cifar10_network.eval()
flops = FlopCountAnalysis(cifar10_network, inputs)
print(flops.total())
# print(flops.by_module())
print(flop_count_table(flops))
# print(flop_count_str(flops))

{'name': 'infer.tiny', 'C': 16, 'N': 5, 'arch_str': '|skip_connect~0|+|skip_connect~0|nor_conv_1x1~1|+|skip_connect~0|skip_connect~1|nor_conv_1x1~2|', 'num_classes': 10}


Unsupported operator aten::add encountered 92 time(s)
Unsupported operator aten::avg_pool2d encountered 2 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
cells.0.layers.0, cells.0.layers.1, cells.0.layers.3, cells.0.layers.4, cells.1.layers.0, cells.1.layers.1, cells.1.layers.3, cells.1.layers.4, cells.10.layers.0, cells.10.layers.1, cells.10.layers.3, cells.10.layers.4, cells.12.layers.0, cells.12.layers.1, cells.12.layers.3, cells.12.layers.4, cells.13.layers.0, cells.13.layers.1, cells.13.layers.3, cells.13.layers.4, cells.14.layers.0, cells.14.layers.1, cells.14.layers.3, cells.14.layers.4, cells.15.layers.0, cells.15.layers.1, cells.15.layers.3, cells.15.layers.4, cells.16.layers.0, cells.16.layers.1, ce

16315008
| module                   | #parameters or shape   | #flops    |
|:-------------------------|:-----------------------|:----------|
| model                    | 0.129M                 | 16.315M   |
|  stem                    |  0.464K                |  0.475M   |
|   stem.0                 |   0.432K               |   0.442M  |
|    stem.0.weight         |    (16, 3, 3, 3)       |           |
|   stem.1                 |   32                   |   32.768K |
|    stem.1.weight         |    (16,)               |           |
|    stem.1.bias           |    (16,)               |           |
|  cells                   |  0.128M                |  15.827M  |
|   cells.0.layers         |   0.576K               |   0.59M   |
|    cells.0.layers.2.op   |    0.288K              |    0.295M |
|    cells.0.layers.5.op   |    0.288K              |    0.295M |
|   cells.1.layers         |   0.576K               |   0.59M   |
|    cells.1.layers.2.op   |    0.288K              |    0.295M |
|

In [103]:
NODE_TYPE_DICT = {
    "none": 0,
    "skip_connect": 1,
    "nor_conv_1x1": 2,
    "nor_conv_3x3": 3,
    "avg_pool_3x3": 4
}

extract_op = lambda item:[NODE_TYPE_DICT[item[0]],item[1]]
opts = api.str2lists(cifar10_config['arch_str'])
print(opts)
# print(len(opts))
cell_opts_list = []
for node_op in opts:
    for op in node_op:
        cell_opts_list.append(extract_op(op))
print(cell_opts_list)
        
N = cifar10_config['N']
cells_idx_list = [i for i in range(N)] + [j+1+N for j in range(N)] + [k+2+2*N for k in range(N)]
print(cells_idx_list)

flops_dict = flops.by_module()
params_dict = parameter_count(cifar10_network)

print(len(cell_opts_list))
for cell_idx in cells_idx_list:
    cell_opt_flops = []
    cell_opt_params = []
    for opt_idx in range(len(cell_opts_list)):
        key = 'cells.{}.layers.{}.op'.format(cell_idx, opt_idx)
        cell_opt_flops.append(flops_dict[key])
        cell_opt_params.append(params_dict[key])
    print(cell_opt_flops)
    print(cell_opt_params)

parameter_count(cifar10_network)['']


[(('skip_connect', 0),), (('skip_connect', 0), ('nor_conv_1x1', 1)), (('skip_connect', 0), ('skip_connect', 1), ('nor_conv_1x1', 2))]
[[1, 0], [1, 0], [2, 1], [1, 0], [1, 1], [2, 2]]
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16]
6
[0, 0, 294912, 0, 0, 294912]
[0, 0, 288, 0, 0, 288]
[0, 0, 294912, 0, 0, 294912]
[0, 0, 288, 0, 0, 288]
[0, 0, 294912, 0, 0, 294912]
[0, 0, 288, 0, 0, 288]
[0, 0, 294912, 0, 0, 294912]
[0, 0, 288, 0, 0, 288]
[0, 0, 294912, 0, 0, 294912]
[0, 0, 288, 0, 0, 288]
[0, 0, 278528, 0, 0, 278528]
[0, 0, 1088, 0, 0, 1088]
[0, 0, 278528, 0, 0, 278528]
[0, 0, 1088, 0, 0, 1088]
[0, 0, 278528, 0, 0, 278528]
[0, 0, 1088, 0, 0, 1088]
[0, 0, 278528, 0, 0, 278528]
[0, 0, 1088, 0, 0, 1088]
[0, 0, 278528, 0, 0, 278528]
[0, 0, 1088, 0, 0, 1088]
[0, 0, 270336, 0, 0, 270336]
[0, 0, 4224, 0, 0, 4224]
[0, 0, 270336, 0, 0, 270336]
[0, 0, 4224, 0, 0, 4224]
[0, 0, 270336, 0, 0, 270336]
[0, 0, 4224, 0, 0, 4224]
[0, 0, 270336, 0, 0, 270336]
[0, 0, 4224, 0, 0, 4224]
[0, 0, 270336, 0,

129306

In [57]:
from thop import profile
import torch

input = torch.randn(1,3,32,32)
flops, params = profile(cifar10_network, inputs={input,})
print(flops/1e6, params/1e6)


from ptflops import get_model_complexity_info
with torch.cuda.device(0):
  macs, params = get_model_complexity_info(cifar10_network, (3, 32, 32), as_strings=False,
                                           print_per_layer_stat=False, verbose=False)
  print('{}  {}'.format('Computational complexity: ', macs))
  print('{}  {}'.format('Number of parameters: ', params))

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register count_avgpool() for <class 'torch.nn.modules.pooling.AvgPool2d'>.
[WARN] Cannot find rule for <class 'xautodl.models.cell_operations.POOLING'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[WARN] Cannot find rule for <class 'xautodl.models.cell_operations.ReLUConvBN'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'xautodl.models.cell_operations.Identity'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.ModuleList'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'xautodl.models.cell_infers.cells.InferCell'>. Treat it as zero M

In [104]:
for network_type in ['cifar10-valid', 'cifar100', 'ImageNet16-120']:
    print(network_type)

cifar10-valid
cifar100
ImageNet16-120


In [105]:
len(api)

15625